<a href="https://colab.research.google.com/github/anonimo-SBBD/SBBD/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import classification_report

import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from xgboost import plot_importance

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [3]:
pd.set_option('display.max_columns', None)

# Predition of suicide rates in the cities

In [ ]:
sul = pd.read_csv('SUL_SIM_ML.csv')
sul

,CODUFMUN,WEEKEND,SEXO_M,SEXO_F,RACACOR_IGN,RACACOR_BRA,RACACOR_PRE,RACACOR_AMA,RACACOR_PAR,RACACOR_IND,ESTCIV_IGN,ESTCIV_SOL,ESTCIV_CAS,ESTCIV_VIU,ESTCIV_SEP,ESTCIV_UNI,ESC_IGN,ESC_NONE,ESC_3ANOS,ESC_7ANOS,ESC_11ANOS,ESC_12MAIS,LOCAL_HOSP,LOCAL_OUTSAU,LOCAL_CASA,LOCAL_VIA,LOCAL_OUTRO,LOCAL_IGN,IDADE_10,IDADE_20,IDADE_30,IDADE_40,IDADE_50,IDADE_60,IDADE_70,IDADE_80,IDADE_90+,SEASON_autumn,SEASON_spring,SEASON_summer,SEASON_winter,CONTADOR,COMPETEN,PROF_SUS,CONSELHO_10.0,CONSELHO_12.0,CONSELHO_15.0,CONSELHO_17.0,CONSELHO_18.0,CONSELHO_19.0,CONSELHO_20.0,CONSELHO_22.0,CONSELHO_23.0,CONSELHO_24.0,CONSELHO_25.0,CONSELHO_26.0,CONSELHO_40.0,CONSELHO_41.0,CONSELHO_42.0,CONSELHO_44.0,CONSELHO_60.0,CONSELHO_61.0,CONSELHO_62.0,CONSELHO_63.0,CONSELHO_64.0,CONSELHO_65.0,CONSELHO_66.0,CONSELHO_67.0,CONSELHO_68.0,CONSELHO_69.0,CONSELHO_70.0,CONSELHO_71.0,CONSELHO_72.0,CONSELHO_73.0,CONSELHO_74.0,CONSELHO_75.0,CONSELHO_76.0,CONSELHO_77.0,CONSELHO_78.0,CONSELHO_79.0,CONSELHO_80.0,CONSELHO_81.0,CONSELHO_82.0,CONSELHO_83.0,CONSELHO_99.0,CONSELHO_0.0,CONSELHO_43.0,TP_UNID_1.0,TP_UNID_2.0,TP_UNID_4.0,TP_UNID_5.0,TP_UNID_7.0,TP_UNID_15.0,TP_UNID_20.0,TP_UNID_21.0,TP_UNID_22.0,TP_UNID_36.0,TP_UNID_39.0,TP_UNID_40.0,TP_UNID_42.0,TP_UNID_43.0,TP_UNID_50.0,TP_UNID_60.0,TP_UNID_61.0,TP_UNID_62.0,TP_UNID_64.0,TP_UNID_67.0,TP_UNID_68.0,TP_UNID_69.0,TP_UNID_70.0,TP_UNID_71.0,TP_UNID_72.0,TP_UNID_73.0,TP_UNID_74.0,TP_UNID_76.0,TP_UNID_77.0,TP_UNID_80.0,TP_UNID_82.0,TP_UNID_83.0,TP_UNID_84.0,SERV_ESP_115.0,HAB_MENTAL,SGRUPHAB_5001.0,SGRUPHAB_6001.0,SUS,SERV_ESP_115.0.1,POP_EST,DENS_DEM,TAXA_SUIC,TAXA_SUICIDIO
0,410010,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,201012.0,39.0,12.0,0.0,22.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,158.0,0.0,0.0,23.0,4.0,86.0,12.0,0.0,0.0,25.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,1.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,7360,33.95,13.586957,HIGH
1,410010,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,201207.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,26.0,0.0,0.0,3.0,2.0,10.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7360,33.95,13.586957,HIGH
2,410010,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,201307.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.0,0.0,0.0,3.0,2.0,9.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7360,33.95,13.586957,HIGH
3,410010,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,201404.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,21.0,0.0,0.0,3.0,2.0,9.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7360,33.95,13.586957,HIGH
4,410010,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,201506.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,19.0,0.0,0.0,3.0,2.0,7.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7360,33.95,13.586957,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

We defined the rates as LOW, MODERATE, or HIGH: rate <= 5 LOW, 5 < rate <= 10 MODERATE, and rate > 10 HIGH.

The classification of based on the World Health Organization.

In [ ]:
sul['TAXA'] = pd.cut(sul['TAXA_SUIC'], labels=False, bins=[0, 5, 10, 150])

## Testing the models

### Train and Test sets

We splitted the train and test sets to train the models and then evaluate the metrics. Then we separed the outcome as the rate and removed the columns with specific data (that is, unique data) or data equivalent to que outcome.

In [ ]:
train, test = train_test_split(sul, test_size= 0.2, random_state= 42)

In [ ]:
y = sul['TAXA']

X = sul.drop(columns= ['TAXA', 'TAXA_SUIC', 'CODUFMUN', 'COMPETEN', 'SERV_ESP_115.0.1', 'WEEKEND', 'TAXA_SUICIDIO', 'CONTADOR', 'POP_EST'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42, stratify= y)

In [ ]:
print(len(train), len(test))

17007 4252


We performed similar analysis for the three models tested, training the models with in a cross validation, separating the dataset into 10 fold, than testing the model and evaluating the metrics of the model and with most important features used by each model.

In [ ]:
# Cross validation
kf = KFold(n_splits=10)

### Decision tree

In [ ]:
dt = DecisionTreeClassifier(max_depth=15)
pipedt = Pipeline([('model', dt)])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipedt.fit(X.loc[train_set], y[train_set])
    sco = pipedt.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.80667921 0.81749765 0.78645343 0.87064911 0.82173095 0.82643462
 0.83725306 0.82079022 0.82031985 0.80894118]
Mean:  0.8216749266781029
SD:  0.020764027371548005


In [ ]:
#Predict the response for test dataset
y_pred_train = pipedt.predict(X_train)
y_pred = pipedt.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 0.9858293643793732
Accuracy: 0.9416745061147695


In [ ]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.98      0.98      0.98      2494
     class 1       0.84      0.86      0.85       760
     class 2       0.93      0.91      0.92       998

    accuracy                           0.94      4252
   macro avg       0.92      0.92      0.92      4252
weighted avg       0.94      0.94      0.94      4252



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[2439   45   10]
 [  53  652   55]
 [   8   77  913]]


In [ ]:
feature_imp = pd.Series(pipedt.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

CONSELHO_71.0    0.430443
DENS_DEM         0.095121
CONSELHO_66.0    0.094838
TP_UNID_70.0     0.048794
PROF_SUS         0.045974
                   ...   
IDADE_20         0.000000
IDADE_10         0.000000
TP_UNID_50.0     0.000000
CONSELHO_79.0    0.000000
TP_UNID_60.0     0.000000
Length: 122, dtype: float64

### Random Forest

In [ ]:
rf = RandomForestClassifier(max_depth=15)
piperf = Pipeline([('model', rf)])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    piperf.fit(X.loc[train_set], y[train_set])
    sco = piperf.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.89604892 0.87629351 0.82690499 0.87300094 0.87158984 0.8593603
 0.86312324 0.8579492  0.89416745 0.86447059]
Mean:  0.8682908970173205
SD:  0.018649430153060458


In [ ]:
y_pred_train = piperf.predict(X_train)
y_pred = piperf.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 0.9967660375139649
Test Accuracy: 0.9459078080903104


In [ ]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.99      0.98      2494
     class 1       0.87      0.83      0.85       760
     class 2       0.94      0.93      0.94       998

    accuracy                           0.95      4252
   macro avg       0.93      0.91      0.92      4252
weighted avg       0.94      0.95      0.95      4252



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[2468   22    4]
 [  81  628   51]
 [   3   69  926]]


In [ ]:
feature_imp = pd.Series(piperf.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

TP_UNID_22.0     0.081317
CONSELHO_71.0    0.077276
CONSELHO_66.0    0.074137
DENS_DEM         0.059899
PROF_SUS         0.052545
                   ...   
TP_UNID_61.0     0.000002
CONSELHO_79.0    0.000001
CONSELHO_0.0     0.000000
CONSELHO_76.0    0.000000
CONSELHO_43.0    0.000000
Length: 122, dtype: float64

### XGBoost

In [ ]:
xg = xgboost.XGBClassifier(learning_rate= 0.1,
                              n_estimators= 3000,
                              subsample= 0.5,
                              colsample_bytree= 0.5,
                              )

pipexg = Pipeline([('model', xg)])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipexg.fit(X.loc[train_set], y[train_set])
    sco = pipexg.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.88805268 0.89087488 0.85042333 0.88617121 0.90592662 0.87817498
 0.88428975 0.90028222 0.89369708 0.86447059]
Mean:  0.8842363344585248
SD:  0.015654297500668636


In [ ]:
y_pred_train = pipexg.predict(X_train)
y_pred = pipexg.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 1.0
Test Accuracy: 0.9724835371589841


In [ ]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.99      0.99      0.99      2494
     class 1       0.93      0.92      0.92       760
     class 2       0.96      0.96      0.96       998

    accuracy                           0.97      4252
   macro avg       0.96      0.96      0.96      4252
weighted avg       0.97      0.97      0.97      4252



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[2480   11    3]
 [  29  697   34]
 [   0   40  958]]


In [ ]:
feature_imp = pd.Series(pipexg.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

TP_UNID_70.0       0.098060
TP_UNID_22.0       0.059876
CONSELHO_71.0      0.039698
PROF_SUS           0.038880
CONSELHO_66.0      0.025971
                     ...   
CONSELHO_76.0      0.000000
CONSELHO_73.0      0.000000
CONSELHO_63.0      0.000000
SGRUPHAB_5001.0    0.000000
CONSELHO_44.0      0.000000
Length: 122, dtype: float32

# Prediction os suicide recurrent attempts in the cities

In [4]:
df = pd.read_csv('Sul_Cidades_ML.csv')
df

,CODUFMUN,COMPETEN,PROF_SUS,CONSELHO_10.0,CONSELHO_12.0,CONSELHO_15.0,CONSELHO_17.0,CONSELHO_18.0,CONSELHO_19.0,CONSELHO_20.0,CONSELHO_22.0,CONSELHO_23.0,CONSELHO_24.0,CONSELHO_25.0,CONSELHO_26.0,CONSELHO_40.0,CONSELHO_41.0,CONSELHO_42.0,CONSELHO_44.0,CONSELHO_60.0,CONSELHO_61.0,CONSELHO_62.0,CONSELHO_63.0,CONSELHO_64.0,CONSELHO_65.0,CONSELHO_66.0,CONSELHO_67.0,CONSELHO_68.0,CONSELHO_69.0,CONSELHO_70.0,CONSELHO_71.0,CONSELHO_72.0,CONSELHO_73.0,CONSELHO_74.0,CONSELHO_75.0,CONSELHO_76.0,CONSELHO_77.0,CONSELHO_78.0,CONSELHO_79.0,CONSELHO_80.0,CONSELHO_81.0,CONSELHO_82.0,CONSELHO_83.0,CONSELHO_99.0,CONSELHO_0.0,CONSELHO_43.0,TP_UNID_1.0,TP_UNID_2.0,TP_UNID_4.0,TP_UNID_5.0,TP_UNID_7.0,TP_UNID_15.0,TP_UNID_20.0,TP_UNID_21.0,TP_UNID_22.0,TP_UNID_36.0,TP_UNID_39.0,TP_UNID_40.0,TP_UNID_42.0,TP_UNID_43.0,TP_UNID_50.0,TP_UNID_60.0,TP_UNID_61.0,TP_UNID_62.0,TP_UNID_64.0,TP_UNID_67.0,TP_UNID_68.0,TP_UNID_69.0,TP_UNID_70.0,TP_UNID_71.0,TP_UNID_72.0,TP_UNID_73.0,TP_UNID_74.0,TP_UNID_76.0,TP_UNID_77.0,TP_UNID_80.0,TP_UNID_82.0,TP_UNID_83.0,TP_UNID_84.0,SERV_ESP_115.0,HAB_MENTAL,SGRUPHAB_5001.0,SGRUPHAB_6001.0,SUS,CD_MUN,MUN,DENS_DEM,SIT_CONJUG_0.0,SIT_CONJUG_1.0,SIT_CONJUG_2.0,SIT_CONJUG_3.0,UF_PR,UF_RS,UF_SC,LOCAL_OCOR_0.0,LOCAL_OCOR_1.0,LOCAL_OCOR_6.0,SEASON_autumn,SEASON_spring,SEASON_summer,SEASON_winter,CIDADE_RESI,OUTRAS_DEFS,OUTROS_AGENTES,CS_SEXO,DEF_TRANS,TRAN_MENT,TRAN_COMP,AG_ENFOR,AG_CORTE,AG_ENVEN,AUTOR_ALCO,OUT_VEZES,WEEKEND,CS_ESCOL_N_0.0,CS_ESCOL_N_1.0,CS_ESCOL_N_2.0,CS_ESCOL_N_3.0,CS_ESCOL_N_4.0,AGE_0.0,AGE_1.0,AGE_2.0,AGE_3.0,AGE_4.0,AGE_5.0,AGE_6.0,AGE_7.0,AGE_8.0,CS_RACA_1.0,CS_RACA_2.0,CS_RACA_3.0,CS_RACA_4.0,CS_RACA_5.0,TAXA_TENT,TAXA
0,410940,201605.0,138.0,0.0,0.0,6.0,60.0,1.0,5.0,0.0,0.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,0.0,0.0,13.0,61.0,0.0,3.0,0.0,4498.0,0.0,0.0,134.0,206.0,5407.0,1.0,0.0,99.0,333.0,0.0,130.0,0.0,0.0,0.0,29.0,13.0,17.0,0.0,0.0,0.0,1.0,44.0,34.0,23.0,1.0,1.0,0.0,0.0,253.0,38.0,27.0,1.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0,2.0,75.0,4109401,GUARAPUAVA,53.68,0,1,1,0,2,0,0,0,2,0,2,0,0,0,2,0,0,2,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,2,0,0,0,0,328.154336,0.0
1,410490,201605.0,34.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,235.0,0.0,0.0,22.0,26.0,207.0,1.0,0.0,8.0,88.0,0.0,14.0,0.0,0.0,0.0,21.0,0.0,12.0,0.0,0.0,0.0,0.0,21.0,1.0,3.0,0.0,0.0,0.0,0.0,61.0,8.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,27.0,4104907,CASTRO,26.50,0,1,1,0,2,0,0,0,2,0,2,0,0,0,2,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,2,0,0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,415.944541,1.0
2,410690,201605.0,858.0,332.0,3.0,896.0,1722.0,611.0,361.0,52.0,48.0,87.0,8.0,0.0,7350.0,3.0,0.0,0.0,0.0,50.0,1244.0,3675.0,0.0,172.0,2.0,176179.0,224.0,14.0,13269.0,9684.0,245468.0,72.0,8.0,3291.0,9063.0,0.0,4946.0,125.0,0.0,141.0,4626.0,1.0,149.0,0.0,0.0,0.0,1.0,391.0,389.0,260.0,54.0,1.0,1.0,2.0,4263.0,335.0,451.0,1.0,29.0,1.0,2.0,5.0,0.0,18.0,0.0,0.0,14.0,1.0,15.0,0.0,2.0,12.0,0.0,1.0,9.0,1.0,1.0,0.0,0.0,0.0,21,8.0,3.0,726.0,4106902,CURITIBA,4027.04,4,19,11,2,36,0,0,1,33,2,36,0,0,0,35,0,3,21,21.0,21.0,0.0,3.0,8.0,26.0,6.0,21.0,9,12,3,4,14,3,10,11,5,5,4,0,1,0,0,24,1,1,5,0,384.218572,1.0
3,411990,201605.0,211.0,29.0,0.0,0.0,102.0,1.0,19.0,1.0,1.0,16.0,7.0,0.0,184.0,1.0,0.0,0.0,0.0,0.0,13.0,218.0,0.0,17.0,0.0,4412.0,5.0,0.0,327.0,339.0,10182.0,5.0,1.0,119.0,650.0,0.0,141.0,0.0,0.0,0.0,1927.0,0.0,114.0,0.0,0.0,0.0,13.0,88.0,10.0,33.0,5.0,0.0,0.0,0.0,623.0,74.0,20.0,3.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0,4.0,132.0,4119905,PONTA GROSSA,150.72,0,4,3,0,7,0,0,0,7,0,7,0,0,0,7,0,7,4,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0,1,2,4,0,1,3,3,0,0,0,0,0,0,7,0,0,0,0,372.591532,1.0
4,412770,201605.0,90.0,4.0,0.0,4.0,25.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0,43.0,0.0,0.0,0.0,2237.0,1.0,0.0,123.0,178.0,2699.0,2.0,0.0,35.0,172

We defined the rates for recurrent suicide attempt in the city as LOW, MODERATE, or HIGH. Since we could not found a classification for this rate, we separated the data into three equal percentages, that is, rate of recurrence <= 33% = LOW, 33% < rate <= 66% = MODERATE, and rate > 66% = HIGH rate of recurrence in the city.


We splitted the train and test sets to train the models and then evaluate the metrics. Then we separed the outcome as the rate and removed the columns with specific data (that is, unique data) or data equivalent to que outcome.

We performed similar analysis for the three models tested, training the models with in a cross validation, separating the dataset into 10 fold, than testing the model and evaluating the metrics of the model and with most important features used by each model.

In [9]:
train, test = train_test_split(df, test_size= 0.2, random_state= 42)

In [10]:
y = df['TAXA']
X = df.drop(columns= ['TAXA', 'TAXA_TENT', 'CODUFMUN', 'COMPETEN', 'CD_MUN', 'MUN', 'OUT_VEZES'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42, stratify= y)

In [11]:
print(len(train), len(test))

8292 2073


In [ ]:
# Cross validation
kf = KFold(n_splits=10)

### Decision tree

In [ ]:
pipedt = Pipeline([('model', DecisionTreeClassifier())])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipedt.fit(X.loc[train_set], y[train_set])
    sco = pipedt.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.94696239 0.95178399 0.92767599 0.95949855 0.95467695 0.96621622
 0.93918919 0.96718147 0.93050193 0.93822394]
Mean:  0.9481910619808402
SD:  0.013356786601671887


In [ ]:
y_pred_train = pipedt.predict(X_train)
y_pred = pipedt.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 1.0
Accuracy: 0.9604438012542209


In [ ]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.72      0.52      0.60       325
     class 1       0.85      0.96      0.90      1588
     class 2       0.90      0.29      0.44       160

    accuracy                           0.84      2073
   macro avg       0.82      0.59      0.65      2073
weighted avg       0.83      0.84      0.82      2073



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[ 168  157    0]
 [  62 1521    5]
 [   4  109   47]]


In [ ]:
feature_imp = pd.Series(pipedt.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

TAXA_TENT        0.262751
DENS_DEM         0.093265
CONSELHO_71.0    0.077527
TP_UNID_2.0      0.073609
CONSELHO_24.0    0.059541
                   ...   
CONSELHO_99.0    0.000000
CONSELHO_83.0    0.000000
CONSELHO_82.0    0.000000
CONSELHO_81.0    0.000000
TP_UNID_68.0     0.000000
Length: 129, dtype: float64

### Random Forest

In [ ]:
piperf = Pipeline([('model', RandomForestClassifier())])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    piperf.fit(X.loc[train_set], y[train_set])
    sco = piperf.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.96721311 0.97974928 0.93828351 0.9710704  0.97203472 0.97779923
 0.97490347 0.98552124 0.95559846 0.96621622]
Mean:  0.9688389622574771
SD:  0.012837674466891637


In [ ]:
y_pred_train = piperf.predict(X_train)
y_pred = piperf.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 1.0
Test Accuracy: 0.9749155812831645


In [ ]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.92      0.94       325
     class 1       0.98      1.00      0.99      1588
     class 2       0.97      0.86      0.91       160

    accuracy                           0.97      2073
   macro avg       0.97      0.92      0.95      2073
weighted avg       0.97      0.97      0.97      2073



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[ 298   23    4]
 [   2 1586    0]
 [   8   15  137]]


In [ ]:
feature_imp = pd.Series(rf.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

TAXA_TENT         0.084486
DENS_DEM          0.045834
CONSELHO_71.0     0.033895
CONSELHO_66.0     0.033608
CONSELHO_75.0     0.028671
                    ...   
AGE_8.0           0.000006
CONSELHO_0.0      0.000000
CONSELHO_41.0     0.000000
SERV_ESP_115.0    0.000000
CONSELHO_43.0     0.000000
Length: 129, dtype: float64

### XGBoost

In [ ]:
pipexg = Pipeline([('model', xgboost.XGBClassifier(learning_rate= 0.02,
                              max_depth= 5,
                              n_estimators= 3000,
                              subsample= 0.5,
                              colsample_bytree= 0.5,
                              eval_metric= 'auc'))])

scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipexg.fit(X.loc[train_set], y[train_set])
    sco = pipexg.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [ ]:
print('Scores: ', np.array(scores))

print('Mean: ', np.array(scores).mean())

print('SD: ', np.array(scores).std())

Scores:  [0.64261047 0.65999436 0.64159853 0.63469129 0.61946716]
Mean:  0.6396723624122794
SD:  0.013102578106703362


In [13]:
y_pred_train = pipexg.predict(X_train)
y_pred = pipexg.predict(X_test)

print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))

Train Accuracy: 0.9990352146647371
Test Accuracy: 0.9657501205981669


In [14]:
y_true = y_test
y_pred = y_pred
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.96      0.90      0.93       325
     class 1       0.97      1.00      0.98      1588
     class 2       0.96      0.79      0.87       160

    accuracy                           0.97      2073
   macro avg       0.96      0.90      0.93      2073
weighted avg       0.97      0.97      0.96      2073



In [15]:
print(confusion_matrix(y_test, y_pred))

[[ 291   30    4]
 [   3 1584    1]
 [  10   23  127]]


In [16]:
feature_imp = pd.Series(pipexg.feature_importances_,index= X.columns).sort_values(ascending=False)
feature_imp

TP_UNID_61.0     0.052450
TP_UNID_62.0     0.024458
UF_RS            0.022003
CONSELHO_78.0    0.016789
CONSELHO_40.0    0.016690
                   ...   
TP_UNID_67.0     0.000000
AGE_8.0          0.000000
CONSELHO_0.0     0.000000
CONSELHO_43.0    0.000000
CS_RACA_3.0      0.000000
Length: 128, dtype: float32